In [1]:
#importing required packages
import os
from email.parser import Parser
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
#initializations
email_body=[]
filename_list=[]
path="C:/Users/prith/Documents/PrithviPython/kamath_prithvi_spring2017/midterm/data/enron/enron_mail_20150507/maildir/lay-k/deleted_items"

#traversing through various directories, subdirectories and files and finding the word 'bankruptcy' in each file.
for directory, subdirectory, filenames in  os.walk(path):
    for filename in filenames:
        with open(os.path.join(directory, filename), "r") as f:
            data = f.read()
        email_body=[]
        email = Parser().parsestr(data)
        email_body.append(email.get_payload())
        for element in email_body:
            for word in element.rstrip().split(' '):
                if word == 'bankruptcy':
                    filename_list.append(directory+"\\"+filename)

In [3]:
#Find the frequency of the word in each file.
filecount={}
for file in filename_list:
    if file not in filecount:
        filecount[file] = 1
    else:
        filecount[file] += 1
sorted_files=sorted(filecount.items(), key=lambda x: x[1], reverse=True)

In [4]:
#count of total number of files
len(filenames)

1126

In [5]:
#count of files which have 'bankruptcy' used twice
len(sorted_files)

1121

In [ ]:
#On checking the output we understand that every file has the word 'bankruptcy' occuring 2 times.
#On closer inspection we understand that body of these emails are same with the only change in salutations.
#Hence we can conclude that there might have been a survey or a poll conducted after the Enron scandle was publisized 
#and the people who signed up for this survey have sent the same mail to My Kenneth Lay